In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cizai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cizai\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Pipeline para la descarga y limpieza de datos del portal de Datos Abiertos

![img](../img/pipeline.png)

In [2]:
## Definimos directorio raiz de donde salen las utilidades "../"
import sys
sys.path.append('../app')
import os
import polars as pd
from sodapy import Socrata
from utils.download_open_data import ContractDataProcessor
from utils.data_cleaner import DataCleaner

## Descarga y actualización de datos bronze

In [3]:
client = Socrata("www.datos.gov.co", None)
processor = ContractDataProcessor(base_path="custom/path/to/data/")

## Limpieza, estandarización y actualización de datos silver

In [4]:
year = 2024
warnings = {}

os.makedirs(f'../Datos/bronze/{year}', exist_ok=True)

# Descargar y procesar los datos de 2024
last_view_2024 = processor.load_last_view_by_year(client, year=2024)

# Guardar los datos en formato parquet
last_view_2024.to_parquet(f'../Datos/bronze/{year}/S2.parquet')

# Mostrar los primeros registros para verificar la descarga
last_view_2024.head()

,nombre_entidad,nit_entidad,departamento,ciudad,orden,sector,rama,entidad_centralizada,proceso_de_compra,id_contrato,...,presupuesto_general_de_la_nacion_pgn,sistema_general_de_participaciones,sistema_general_de_regal_as,recursos_propios_alcald_as_gobernaciones_y_resguardos_ind_genas_,recursos_de_credito,recursos_propios,ultima_actualizacion,codigo_entidad,codigo_proveedor,objeto_del_contrato
0,ESTADO SURORIENTE ESE,9001455729,Cauca,Popayán,Territorial,Salud y Protección Social,Corporación Autónoma,Centralizada,CO1.BDOS.5361409,CO1.PCCNTR.5707469,...,0,0,0,0,0,14912640,2024-03-30T00:00:00.000,718384050,705083525,PRESTACIÓN DE SERVICIOS PROFESIONALES DE APOYO...
1,HOSPITAL SAN JUAN DE DIOS DE HONDA ESE,890700666,Tolima,Honda,Territorial,Salud y Protección Social,Corporación Autónoma,Centralizada,CO1.BDOS.5355863,CO1.PCCNTR.5705317,...,0,46398000,0,0,0,0,<NA>,703294033,716217096,EJECUTAR LOS PROCESOS Y SUBPROCESOS DE PSICOLO...
2,EMPRESA SOCIAL DEL ESTADO HOSPITAL LOCAL DE SA...,892000458,Meta,San Martín,Nacional,Salud y Protección Social,Ejecutivo,Centralizada,CO1.BDOS.5360776,CO1.PCCNTR.5708015,...,0,0,0,0,0,3059636,2024-02-25T00:00:00.000,705985851,719332090,PRESTACIÓN DE SERVICIOS PROFESIONALES COMO BAC...
3,HOSPITAL DEPARTAMENTAL UNIVERSITARIO SANTA SOF...,890801099,Caldas,Manizales,Territorial,Salud y Protección Social,Ejecutivo,Centralizada,CO1.BDOS.5326534,CO1.PCCNTR.5688535,...,0,0,0,45000000,0,0,<NA>,704527647,720182872,PRESTACION DE SERVICIOS PROFESIONALES COMO FIS...
4,empresa social del estado región de salud soacha.,8000068503,Cundinamarca,Soacha,Territorial,Salud y Protección Social,Corporación Autónoma,Centralizada,CO1.BDOS.5357885,CO1.PCCNTR.5705929,...,9336000,0,0,0,0,0,<NA>,702727421,720042449,PRESTACIÓN DE SERVICIOS COMO PROFESIONAL EN SA...


In [5]:
# Instanciar la clase DataCleaner
cleaner = DataCleaner()

# Ruta de entrada del archivo Parquet que se desea limpiar
input_path = '../Datos/bronze/2024/S2.parquet'  # Ajusta esta ruta según la ubicación de tu archivo

# Cargar los datos usando DataCleaner (cambiando file_type a 'parquet')
# Intentar con el encoding 'ISO-8859-1' o 'latin1'
df = cleaner.load_data(file_path=input_path, file_type='parquet')

# Definir las columnas a procesar (ajusta estas listas a tu dataset)
columns_to_cast = ['nombre_entidad', 'nit_entidad', 'departamento', 'ciudad']  # Ajusta según tu DataFrame
categorical_columns = ['estado_contrato', 'habilita_pago_adelantado']  # Ajusta según tu DataFrame
text_columns = ['descripcion_del_proceso', 'justificacion_modalidad_de', 'objeto_del_contrato']  # Ajusta según tu DataFrame
numeric_columns = ['saldo_cdp', 'saldo_vigencia']  # Ajusta según tu DataFrame
date_columns = ['fecha_de_firma', 'fecha_de_inicio_del_contrato']  # Ajusta según tu DataFrame
url_columns = ['urlproceso']  # Ajusta según tu DataFrame

# Paso 1: Castear columnas a string
df = cleaner.cast_columns(df, columns_to_cast=columns_to_cast)

# Paso 2: Codificar columnas categóricas
df = cleaner.encode_categorical_columns(df, categorical_columns=categorical_columns)

# Paso 3: Limpiar texto en columnas de tipo texto
df = cleaner.clean_text_columns(df, text_columns=text_columns)

# Paso 4: Limpiar valores numéricos en columnas numéricas
df = cleaner.clean_numeric_columns(df, numeric_columns=numeric_columns)

# Paso 5: Limpiar y estandarizar columnas de fechas
df = cleaner.clean_date_columns(df, date_columns=date_columns)

# Paso 6: Limpiar URLs en columnas específicas
df = cleaner.clean_url_columns(df, url_columns=url_columns)

# Paso 7: Limpiar caracteres especiales en columnas de descripción
df = cleaner.clean_descriptions_columns(df, cols=text_columns)

# Paso 8: Guardar el DataFrame limpio en formato Parquet
output_path = '../Datos/bronze/2024/S2_cleaned.parquet'  # Ajusta esta ruta según tu entorno
cleaner.save_data(df, output_path=output_path, file_type='parquet')

# Mostrar los primeros registros para verificar la limpieza
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cizai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cizai\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\cizai\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


                                      nombre_entidad nit_entidad  \
0                              ESTADO SURORIENTE ESE  9001455729   
1             HOSPITAL SAN JUAN DE DIOS DE HONDA ESE   890700666   
2  EMPRESA SOCIAL DEL ESTADO HOSPITAL LOCAL DE SA...   892000458   
3  HOSPITAL DEPARTAMENTAL UNIVERSITARIO SANTA SOF...   890801099   
4  empresa social del estado región de salud soacha.  8000068503   

   departamento      ciudad        orden                     sector  \
0         Cauca     Popayán  Territorial  Salud y Protección Social   
1        Tolima       Honda  Territorial  Salud y Protección Social   
2          Meta  San Martín     Nacional  Salud y Protección Social   
3        Caldas   Manizales  Territorial  Salud y Protección Social   
4  Cundinamarca      Soacha  Territorial  Salud y Protección Social   

                   rama entidad_centralizada proceso_de_compra  \
0  Corporación Autónoma         Centralizada  CO1.BDOS.5361409   
1  Corporación Autónoma         